In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

# Created config.py file which will have store credentials of PgAdmin
from config import PgAdmin_userid, PgAdmin_password


# Abstract :

We are creating a database on the World Happiness scores and pairing that data with data from World Bank to see which data points affect the Happiness scores (e.g. population, corruption, GDP per capita etc.)

# 1. Extract : 

We collected the data from kaggle.com. The data was in .csv format. Below are the link of our dataset:

1) https://www.kaggle.com/mathurinache/world-happiness-report?select=2020.csv <br/>
2) https://www.kaggle.com/mathurinache/world-happiness-report?select=2019.csv <br/>
3) https://www.kaggle.com/PromptCloudHQ/world-happiness-report-2019 <br/>
4) https://www.kaggle.com/tanuprabhu/population-by-country-2020


### Extract CSVs into DataFrames

In [22]:
# Import the .csv files
gdp_csv_file = "Resources/happiness_index.csv"
GDP_index_df = pd.read_csv(gdp_csv_file)
GDP_index_df.head()


,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.8087,0.031156,7.869766,7.747634,10.639267,0.954330,71.900825,0.949172,-0.059482,0.195445,1.972317,1.285190,1.499526,0.961271,0.662317,0.159670,0.477857,2.762835
1,Denmark,Western Europe,7.6456,0.033492,7.711245,7.579955,10.774001,0.955991,72.402504,0.951444,0.066202,0.168489,1.972317,1.326949,1.503449,0.979333,0.665040,0.242793,0.495260,2.432741
2,Switzerland,Western Europe,7.5599,0.035014,7.628528,7.491272,10.979933,0.942847,74.102448,0.921337,0.105911,0.303728,1.972317,1.390774,1.472403,1.040533,0.628954,0.269056,0.407946,2.350267
3,Iceland,Western Europe,7.5045,0.059616,7.621347,7.387653,10.772559,0.974670,73.000000,0.948892,0.246944,0.711710,1.972317,1.326502,1.547567,1.000843,0.661981,0.362330,0.144541,2.460688
4,Norway,Western Europe,7.4880,0.034837,7.556281,7.419719,11.087804,0.952487,73.200783,0.955750,0.134533,0.263218,1.972317,1.424207,1.495173,1.008072,0.670201,0.287985,0.434101,2.168266


In [3]:
# Import the .csv files
happiness_csv_file = "Resources/happiness_index2019.csv"
happiness_index_df = pd.read_csv(happiness_csv_file)
happiness_index_df


,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
...,...,...,...,...,...,...,...,...,...
151,152,Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411
152,153,Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147
153,154,Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025
154,155,Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035


In [4]:
# Import the .csv files
corruption_csv_file = "Resources/corruption_index.csv"
corruption_index_df = pd.read_csv(corruption_csv_file)
corruption_index_df


,Country (region),Ladder,SD of Ladder,Positive affect,Negative affect,Social support,Freedom,Corruption,Generosity,Log of GDP\nper capita,Healthy life\nexpectancy
0,Finland,1,4,41.0,10.0,2.0,5.0,4.0,47.0,22.0,27.0
1,Denmark,2,13,24.0,26.0,4.0,6.0,3.0,22.0,14.0,23.0
2,Norway,3,8,16.0,29.0,3.0,3.0,8.0,11.0,7.0,12.0
3,Iceland,4,9,3.0,3.0,1.0,7.0,45.0,3.0,15.0,13.0
4,Netherlands,5,1,12.0,25.0,15.0,19.0,12.0,7.0,12.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...
151,Rwanda,152,63,54.0,102.0,144.0,21.0,2.0,90.0,132.0,103.0
152,Tanzania,153,122,78.0,50.0,131.0,78.0,34.0,49.0,125.0,118.0
153,Afghanistan,154,25,152.0,133.0,151.0,155.0,136.0,137.0,134.0,139.0
154,Central African Republic,155,117,132.0,153.0,155.0,133.0,122.0,113.0,152.0,150.0


In [5]:
# Import the .csv files
population_csv_file = "Resources/population.csv"
population_index_df = pd.read_csv(population_csv_file)
population_index_df


,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,China,1438207241,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,India,1377233523,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,United States,330610570,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,Indonesia,272931713,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,Pakistan,219992900,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %
...,...,...,...,...,...,...,...,...,...,...,...
230,Montserrat,4991,0.06 %,3,50,100,NaN,N.A.,N.A.,10 %,0.00 %
231,Falkland Islands,3458,3.05 %,103,0,12170,NaN,N.A.,N.A.,66 %,0.00 %
232,Niue,1624,0.68 %,11,6,260,NaN,N.A.,N.A.,46 %,0.00 %
233,Tokelau,1354,1.27 %,17,136,10,NaN,N.A.,N.A.,0 %,0.00 %


# 2. Transform :

The dataset we got was clean so we were not required to drop any data points like null data. However, we filtered the DataFrame and extracted specific columns that we wanted to keep in our database and renamed the columns' headers to enable merging of two DataFrames based on 'Country' column. Lastly we renamed the column headers to match the column header in our schema.

### Transform to DataFrame

In [6]:
# Create a filtered dataframe from specific columns
GDP_data_df = GDP_index_df[['Country name', 'Logged GDP per capita']].copy()

# Rename the column headers
GDP_data_df = GDP_data_df.rename(columns = {"Country name" : "Country",
                                         "Logged GDP per capita" : "GDP per capita"
                                         })
GDP_data_df.head()


,Country,GDP per capita
0,Finland,10.639267
1,Denmark,10.774001
2,Switzerland,10.979933
3,Iceland,10.772559
4,Norway,11.087804


In [7]:
# Create a filtered dataframe from specific columns
happiness_df = happiness_index_df[['Overall rank', "Country or region",  'Score']].copy()

# Rename the column headers
happiness_df = happiness_df.rename(columns = {"Overall rank" : "Happiness Rank", 
                                              "Country or region" : "Country",
                                              "Score" : "Happiness Score"
                                         })
happiness_df.head()


,Happiness Rank,Country,Happiness Score
0,1,Finland,7.769
1,2,Denmark,7.600
2,3,Norway,7.554
3,4,Iceland,7.494
4,5,Netherlands,7.488


In [8]:
# Merge the DataFrames
merge_one = pd.merge(left = GDP_data_df, right = happiness_df, how = "inner", on = "Country" )
merge_one


,Country,GDP per capita,Happiness Rank,Happiness Score
0,Finland,10.639267,1,7.769
1,Denmark,10.774001,2,7.600
2,Switzerland,10.979933,6,7.480
3,Iceland,10.772559,4,7.494
4,Norway,11.087804,3,7.554
...,...,...,...,...
142,Central African Republic,6.625160,155,3.083
143,Rwanda,7.600104,152,3.334
144,Zimbabwe,7.865712,146,3.663
145,South Sudan,7.425360,156,2.853


In [9]:
# Create a filtered dataframe from specific columns
corruption_df = corruption_index_df[["Country (region)", "Corruption"]].copy()

# Rename the column headers
corruption_df = corruption_df.rename(columns = {"Country (region)" : "Country", 
                                              "Corruption" : "Corruption Score"
                                         })
corruption_df.head()


,Country,Corruption Score
0,Finland,4.0
1,Denmark,3.0
2,Norway,8.0
3,Iceland,45.0
4,Netherlands,12.0


In [10]:
# Merge the DataFrames
merge_two = pd.merge(left = merge_one, right = corruption_df, how = "inner", on = "Country" )
merge_two


,Country,GDP per capita,Happiness Rank,Happiness Score,Corruption Score
0,Finland,10.639267,1,7.769,4.0
1,Denmark,10.774001,2,7.600,3.0
2,Switzerland,10.979933,6,7.480,7.0
3,Iceland,10.772559,4,7.494,45.0
4,Norway,11.087804,3,7.554,8.0
...,...,...,...,...,...
141,Central African Republic,6.625160,155,3.083,122.0
142,Rwanda,7.600104,152,3.334,2.0
143,Zimbabwe,7.865712,146,3.663,63.0
144,South Sudan,7.425360,156,2.853,61.0


In [11]:
# Create a filtered dataframe from specific columns
population_df = population_index_df[["Country (or dependency)", "Population (2020)", "Density (P/Km²)"]]

# Rename the column headers
population_df  = population_df .rename(columns = {"Country (or dependency)" : "Country", 
                                              "Population (2020)" : "Population"
                                         })
population_df.head()


,Country,Population,Density (P/Km²)
0,China,1438207241,153
1,India,1377233523,464
2,United States,330610570,36
3,Indonesia,272931713,151
4,Pakistan,219992900,287


In [12]:
# Merge the DataFrames to get the final DataFrame on which we will do Data Modelling
combined_df = pd.merge(left = merge_two, right = population_df, how = "inner", on = "Country" )
combined_df


,Country,GDP per capita,Happiness Rank,Happiness Score,Corruption Score,Population,Density (P/Km²)
0,Finland,10.639267,1,7.769,4.0,5539002,18
1,Denmark,10.774001,2,7.600,3.0,5788108,137
2,Switzerland,10.979933,6,7.480,7.0,8641786,219
3,Iceland,10.772559,4,7.494,45.0,340795,3
4,Norway,11.087804,3,7.554,8.0,5412632,15
...,...,...,...,...,...,...,...
134,Central African Republic,6.625160,155,3.083,122.0,4812256,8
135,Rwanda,7.600104,152,3.334,2.0,12883878,525
136,Zimbabwe,7.865712,146,3.663,63.0,14818157,38
137,South Sudan,7.425360,156,2.853,61.0,11166783,18


In [13]:
# Reset the index
new_combined_df = combined_df.reset_index()
new_combined_df["index"] += 1
new_combined_df = new_combined_df.rename(columns = {"index" : "Country ID"})
new_combined_df


,Country ID,Country,GDP per capita,Happiness Rank,Happiness Score,Corruption Score,Population,Density (P/Km²)
0,1,Finland,10.639267,1,7.769,4.0,5539002,18
1,2,Denmark,10.774001,2,7.600,3.0,5788108,137
2,3,Switzerland,10.979933,6,7.480,7.0,8641786,219
3,4,Iceland,10.772559,4,7.494,45.0,340795,3
4,5,Norway,11.087804,3,7.554,8.0,5412632,15
...,...,...,...,...,...,...,...,...
134,135,Central African Republic,6.625160,155,3.083,122.0,4812256,8
135,136,Rwanda,7.600104,152,3.334,2.0,12883878,525
136,137,Zimbabwe,7.865712,146,3.663,63.0,14818157,38
137,138,South Sudan,7.425360,156,2.853,61.0,11166783,18


In [14]:
# Rename the column headers to match the column header in our schema
new_combined_df = new_combined_df.rename(columns = {"Country ID" : "country_id",
                                                   "Country" : "country_name",
                                                   "GDP per capita" : "gdp_per_capita",
                                                   "Happiness Rank" : "happiness_rank",
                                                   "Happiness Score" : "happiness_score",
                                                   "Corruption Score" : "corruption_score",
                                                   "Population" : "population",
                                                   "Density (P/Km²)" : "density"
                                                   })


new_combined_df.head()


,country_id,country_name,gdp_per_capita,happiness_rank,happiness_score,corruption_score,population,density
0,1,Finland,10.639267,1,7.769,4.0,5539002,18
1,2,Denmark,10.774001,2,7.600,3.0,5788108,137
2,3,Switzerland,10.979933,6,7.480,7.0,8641786,219
3,4,Iceland,10.772559,4,7.494,45.0,340795,3
4,5,Norway,11.087804,3,7.554,8.0,5412632,15


# 3. Load :

We had a structured dataset so we decided to use PostgresSql to create the database.

In PgAdmin, we created a new database named 'ETL_project_db' and created tables inside this database. For connecting to this database, we used 'create_engine' module from SQLAlchemy. Finally, we did loading of DataFrame into the SQL database.

### Create database connection

In [15]:
# Create the connection to engine
# The database name is "ETL_project_db"
if PgAdmin_password == None:
    
    # If only PgAdmin User ID is available, then connection will be...
    connection_string = f"{PgAdmin_userid}@localhost:5432/ETL_project_db"
    
else:
        
    # If both PgAdmin User ID and Password are available, then connection will be...
    connection_string = f"{PgAdmin_userid}:{PgAdmin_password}@localhost:5432/ETL_project_db"
            
engine = create_engine(f"postgresql://{connection_string}")


In [16]:
# Confirm tables which we created in SQL Database
engine.table_names()


['country', 'happiness_index', 'corruption_index', 'population', 'gdp_data']

### Load DataFrame into database

In [17]:
# Load DataFrame to 'country' table in SQL Database
new_combined_df[["country_id", "country_name"]].to_sql(name="country", con = engine, if_exists="append", index=False)
    

In [18]:
# Load DataFrame to 'happiness_score' table in SQL Database
new_combined_df[["happiness_rank", "country_id", "happiness_score"]].to_sql(name="happiness_index", con = engine, if_exists="append", index=False)


In [19]:
# Load DataFrame to 'corruption_index' table in SQL Database
new_combined_df[["country_id", "corruption_score"]].to_sql(name="corruption_index", con = engine, if_exists="append", index=False)


In [20]:
# Load DataFrame to 'population' table in SQL Database
new_combined_df[["country_id", "population", "density"]].to_sql(name="population", con = engine, if_exists="append", index=False)


In [21]:
# Load DataFrame to 'gdp_data' table in SQL Database
new_combined_df[["country_id", "gdp_per_capita"]].to_sql(name="gdp_data", con = engine, if_exists="append", index=False)
